# Tutorial 5, solutions


This solution is a jupyter notebook which allows you to directly interact with the code so that
you can see the effect of any changes you may like to make.

Author: Nicky van Foreest

In [1]:
from collections import deque
from heapq import heappop, heappush
import numpy as np
from scipy.stats import expon, uniform

np.random.seed(8)

ARRIVAL = 0
DEPARTURE = 1

ECONOMY = 0
BUSINESS = 1

In [2]:


class Job:
    def __init__(self):
        self.arrival_time = 0
        self.service_time = 0
        self.customer_type = ECONOMY
        self.server_type = ECONOMY
        self.departure_time = 0
        self.queue_length_at_arrival = 0

    def sojourn_time(self):
        return self.departure_time - self.arrival_time

    def waiting_time(self):
        return self.sojourn_time() - self.service_time

    def service_start(self):
        return self.departure_time - self.service_time

    def __repr__(self):
        return f"{self.customer_type}, {self.server_type}, {self.arrival_time}, {self.service_time}, {self.service_start()}, {self.departure_time}\n"

    def __lt__(self, other):
        # this is necessary to sort jobs when they have the same arrival times.
        return self.queue_length_at_arrival < other.queue_length_at_arrival


In [3]:


def generate_jobs(A, S, p_business):
    jobs = set()
    num_jobs = len(A)
    p = uniform(0, 1).rvs(num_jobs)

    for n in range(num_jobs):
        job = Job()
        job.arrival_time = A[n]
        job.service_time = S[n]
        if p[n] < p_business:
            job.customer_type = BUSINESS
        else:
            job.customer_type = ECONOMY
        jobs.add(job)

    return jobs


In [4]:


class GGc_with_business:
    def __init__(self, c, jobs):
        self.b = 1  # number of b servers
        self.c = c  # number of e servers
        self.jobs = jobs

        self.num_b_busy = 0
        self.num_e_busy = 0
        self.stack = []
        self.b_queue = deque()
        self.e_queue = deque()

        self.fill_stack()

    def fill_stack(self):
        for job in sorted(self.jobs, key=lambda j: j.arrival_time):
            heappush(self.stack, (job.arrival_time, job, ARRIVAL))

    def handle_arrival(self, time, job):
        if job.customer_type == BUSINESS:
            job.queue_length_at_arrival = len(self.b_queue)
        else:
            job.queue_length_at_arrival = len(self.e_queue)

        if job.customer_type == ECONOMY:
            if self.num_e_busy < self.c:
                job.server_type = ECONOMY
                self.start_service(time, job)
            elif self.num_b_busy < self.b:
                job.server_type = BUSINESS
                self.start_service(time, job)
            else:
                self.e_queue.append(job)
        else:  # business customer
            if self.num_b_busy < self.b:
                job.server_type = BUSINESS
                self.start_service(time, job)
            elif self.num_e_busy < self.c:
                job.server_type = ECONOMY
                self.start_service(time, job)
            else:
                self.b_queue.append(job)

    def start_service(self, time, job):
        if job.server_type == BUSINESS:
            self.num_b_busy += 1
        else:
            self.num_e_busy += 1
        job.departure_time = time + job.service_time
        heappush(self.stack, (job.departure_time, job, DEPARTURE))

    def pop_from_queue_set_server_and_start(self, time, queue, server_type):
        next_job = queue.popleft()
        next_job.server_type = server_type
        self.start_service(time, next_job)

    def handle_departure(self, time, job):
        if job.server_type == BUSINESS:
            self.num_b_busy -= 1
            if self.b_queue:
                self.pop_from_queue_set_server_and_start(time, self.b_queue, BUSINESS)
            elif self.e_queue:
                self.pop_from_queue_set_server_and_start(time, self.e_queue, BUSINESS)
        else:  # economy server free
            self.num_e_busy -= 1
            if self.e_queue:
                self.pop_from_queue_set_server_and_start(time, self.e_queue, ECONOMY)
            elif self.b_queue:
                self.pop_from_queue_set_server_and_start(time, self.b_queue, ECONOMY)

    def run(self):
        time = 0
        while self.stack:  # not empty
            time, job, epoch_type = heappop(self.stack)
            if epoch_type == ARRIVAL:
                self.handle_arrival(time, job)
            else:
                self.handle_departure(time, job)

    def print_served_job(self):
        for j in sorted(self.jobs, key=lambda j: j.arrival_time):
            print(j)

    def mean_waiting_time(self, customer_type=None):
        if customer_type is None:
            jobs = self.jobs
        else:
            jobs = set(j for j in self.jobs if j.customer_type == customer_type)
        return sum(j.waiting_time() for j in jobs) / len(jobs)

    def max_waiting_time(self, customer_type=None):
        if customer_type is None:
            return max(j.waiting_time() for j in self.jobs)
        else:
            return max(
                j.waiting_time() for j in self.jobs if j.customer_type == customer_type
            )


In [5]:
def sakasegawa(F, G, c):
    labda = 1.0 / F.mean()
    ES = G.mean()
    rho = labda * ES / c
    EWQ_1 = rho ** (np.sqrt(2 * (c + 1)) - 1) / (c * (1 - rho)) * ES
    ca2 = F.var() * labda * labda
    ce2 = G.var() / ES / ES
    return (ca2 + ce2) / 2 * EWQ_1


In [6]:


def make_arrivals_and_services(F, G, num_jobs):
    a = F.rvs(num_jobs)
    A = np.cumsum(a)
    S = G.rvs(num_jobs)
    return A, S


In [7]:


def DD1_test_1():
    # test with only business customers
    c = 0
    F = uniform(1, 0.0001)
    G = expon(0.5, 0.0001)
    num_jobs = 5
    p_business = 1
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()
    ggc.print_served_job()


DD1_test_1()

1, 1, 1.0000873429402792, 0.5000011464268599, 1.0000873429402792, 1.5000884893671391

1, 1, 2.0001841970065612, 0.5000562941744815, 2.0001841970065612, 2.5002404911810427

1, 1, 3.0002711164605826, 0.5000514752256303, 3.0002711164605826, 3.500322591686213

1, 1, 4.000324202029738, 0.5000739556989864, 4.000324202029738, 4.500398157728724

1, 1, 5.000347474862536, 0.5000650838539586, 5.000347474862536, 5.500412558716494



In [8]:


def DD1_test_2():
    # test with only economy customers
    c = 1
    F = uniform(1, 0.0001)
    G = expon(0.5, 0.0001)
    p_business = 0
    num_jobs = 5
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()
    ggc.print_served_job()


DD1_test_2()

0, 0, 1.0000426091770476, 0.5000068073709532, 1.0000426091770476, 1.5000494165480007

0, 0, 2.0000715166798493, 0.5004066955873369, 2.0000715166798493, 2.5004782122671863

0, 0, 3.0001689022039693, 0.5000136800401932, 3.0001689022039693, 3.5001825822441623

0, 0, 4.00020227960852, 0.5000388800919982, 4.00020227960852, 4.5002411597005185

0, 0, 5.000224159714602, 0.5000073585018648, 5.000224159714602, 5.5002315182164665



In [9]:


def DD1_test_3():
    # test with only economy customers but only a business server
    c = 0
    F = uniform(1, 0.0001)
    G = expon(0.5, 0.0001)
    p_business = 0
    num_jobs = 5
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()
    ggc.print_served_job()


DD1_test_3()

0, 1, 1.0000028732048962, 0.5000385206403165, 1.0000028732048962, 1.5000413938452128

0, 1, 2.0000380416785233, 0.5000566472121346, 2.0000380416785237, 2.500094688890658

0, 1, 3.0000761375674436, 0.5000314910410787, 3.0000761375674436, 3.5001076286085224

0, 1, 4.00015255358058, 0.500161473132951, 4.00015255358058, 4.500314026713531

0, 1, 5.0002464339719115, 0.5001016770970268, 5.0002464339719115, 5.500348111068938



In [10]:


def DD2_test_1():
    # test with only economy customers and one e_server. As the b_server is always present, we must have 2 servers.
    # assume that all jobs arrive at time 0, and have service time 1
    c = 1
    F = uniform(0, 0.0001)
    G = expon(1, 0.0001)
    p_business = 0
    num_jobs = 10
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()
    ggc.print_served_job()


DD2_test_1()

0, 0, 7.901725413382857e-05, 1.0000105790379397, 7.901725413383964e-05, 1.0000895962920735

0, 1, 0.0001778769059452015, 1.000131363303936, 0.0001778769059452845, 1.0003092402098812

0, 0, 0.00023627336029529816, 1.0002807611555136, 1.0000895962920737, 2.0003703574475873

0, 1, 0.00024016459005953036, 1.0002870301833324, 1.000309240209881, 2.0005962703932134

0, 0, 0.0002848068808113206, 1.0000469585324776, 2.0003703574475873, 3.000417315980065

0, 1, 0.00030362413234065206, 1.0000905722388749, 2.0005962703932134, 3.0006868426320885

0, 0, 0.0003663296970261769, 1.000108465823769, 3.000417315980065, 4.000525781803834

0, 1, 0.0003878614769734515, 1.0000714535192632, 3.0006868426320885, 4.000758296151352

0, 0, 0.00040295848773920116, 1.0000636563808463, 4.000525781803834, 5.00058943818468

0, 1, 0.0004565729184071586, 1.0000008911878673, 4.000758296151352, 5.000759187339219



In [11]:


def mm1_test_1():
    # test with only business customers but no e_server, very few jobs
    c = 0
    labda = 0.9
    mu = 1
    F = expon(scale=1.0 / labda)
    G = expon(scale=1.0 / mu)
    p_business = 1

    num_jobs = 10
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()
    ggc.print_served_job()


mm1_test_1()

1, 1, 0.9829148402935132, 0.935667583156166, 0.9829148402935131, 1.918582423449679

1, 1, 2.34505995130281, 1.7089023830468935, 2.3450599513028103, 4.053962334349704

1, 1, 3.1389920101377657, 0.750191376400479, 4.053962334349704, 4.804153710750183

1, 1, 5.108236031750993, 2.8002335190323526, 5.108236031750993, 7.908469550783345

1, 1, 6.525336122731557, 1.2851229012716343, 7.9084695507833445, 9.193592452054979

1, 1, 8.760747506808187, 0.19693625188996403, 9.193592452054979, 9.390528703944943

1, 1, 9.013430701959255, 0.9668616992971492, 9.390528703944943, 10.357390403242093

1, 1, 9.407412992806286, 0.826183573343697, 10.357390403242093, 11.183573976585789

1, 1, 10.12506817157068, 1.4414038315889446, 11.183573976585789, 12.624977808174734

1, 1, 10.881109605285141, 1.6432343069487785, 12.624977808174734, 14.268212115123513



In [12]:


def mm1_test_2():
    # test with only economy customers but no e_server
    c = 0
    labda = 0.9
    mu = 1
    F = expon(scale=1.0 / labda)
    G = expon(scale=1.0 / mu)
    p_business = 0

    print("theory: ", sakasegawa(F, G, c + 1))  # 1 for the business server

    num_jobs = 100_000
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()

    print("mean waiting: ", ggc.mean_waiting_time())


mm1_test_2()

theory:  8.999999999999991
mean waiting:  8.717092494163326


In [13]:


def mm1_test_3():
    # test with only business customers but no e_server
    c = 0
    labda = 0.9
    mu = 1
    F = expon(scale=1.0 / labda)
    G = expon(scale=1.0 / mu)
    p_business = 1

    print("theory: ", sakasegawa(F, G, c + 1))  # 1 for the business server

    num_jobs = 100_000
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()

    print("mean waiting: ", ggc.mean_waiting_time())


mm1_test_3()

theory:  8.999999999999991
mean waiting:  9.736027176700428


In [14]:


def mm2_test_1():
    # test with only business customers and 1 e_server
    c = 1
    labda = 0.9
    mu = 1
    F = expon(scale=1.0 / labda)
    G = expon(scale=1.0 / mu)
    p_business = 1

    print("theory: ", sakasegawa(F, G, c + 1))  # 1 for the business server

    num_jobs = 100_000
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()

    # mind that Sakasegawa's result is an approximation for the M/M/c with c>1
    print("mean waiting: ", ggc.mean_waiting_time())


mm2_test_1()


theory:  0.28572116393706454
mean waiting:  0.2624273514424324


In [15]:


def mm2_test_2():
    # test with only economy customers and 1 e_server
    c = 1
    labda = 0.9
    mu = 1
    F = expon(scale=1.0 / labda)
    G = expon(scale=1.0 / mu)
    p_business = 0

    print("theory: ", sakasegawa(F, G, c + 1))  # 1 for the business server

    num_jobs = 100_000
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    ggc = GGc_with_business(c, jobs)
    ggc.run()

    print("mean waiting: ", ggc.mean_waiting_time())


mm2_test_2()
    

theory:  0.28572116393706454
mean waiting:  0.2617081708185768


In [16]:
import copy  # to copy the simulation data


def case_analysis(jobs, c):
    # we need the same jobs for all cases, so that we can compare in a fair way.
    b_jobs = set(copy.copy(j) for j in jobs if j.customer_type == BUSINESS)
    e_jobs = set(copy.copy(j) for j in jobs if j.customer_type == ECONOMY)

    # Case 1: each class its own server, no sharing
    bus = GGc_with_business(0, b_jobs)
    bus.run()

    eco = GGc_with_business(c - 1, e_jobs)
    eco.run()

    # Case 2: sharing with business server
    shared = GGc_with_business(c, jobs)
    shared.run()

    print("separate: bus mean", bus.mean_waiting_time())
    print("shared: bus mean: ", shared.mean_waiting_time(BUSINESS))
    print("separate: bus max", bus.max_waiting_time())
    print("shared: bus max: ", shared.max_waiting_time(BUSINESS))

    print("separate: eco mean", eco.mean_waiting_time())
    print("shared: eco mean: ", shared.mean_waiting_time(ECONOMY))
    print("separate: eco max", eco.max_waiting_time())
    print("shared: eco max: ", shared.max_waiting_time(ECONOMY))

    print("shared: all mean: ", shared.mean_waiting_time())
    print("shared: all max: ", shared.max_waiting_time())
    print()


In [17]:


def case1():
    num_jobs = 300
    opening_time_of_desks = 60  # minutes
    labda = num_jobs / opening_time_of_desks
    F = expon(scale=1.0 / labda)
    G = uniform(1, 2)
    p_business = 0.1
    c = 6
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    case_analysis(jobs, c)


case1()

separate: bus mean 2.620893590927028
shared: bus mean:  4.307184870016928
separate: bus max 9.757114669693307
shared: bus max:  11.170118916403565
separate: eco mean 16.194420279720028
shared: eco mean:  14.5886901390232
separate: eco max 30.126955061480285
shared: eco max:  26.081439961405717
shared: all mean:  13.629082980582611
shared: all max:  26.081439961405717



In [18]:


def case2():
    num_jobs = 300
    labda = num_jobs / 180
    F = expon(scale=1.0 / labda)
    G = uniform(1, 2)
    p_business = 0.05
    c = 5
    A, S = make_arrivals_and_services(F, G, num_jobs)
    jobs = generate_jobs(A, S, p_business)
    case_analysis(jobs, c)


case2()
      


separate: bus mean 0.23674610086377795
shared: bus mean:  0.13773541717589446
separate: bus max 1.9737835833394382
shared: bus max:  1.6670768334516914
separate: eco mean 0.22297372849376582
shared: eco mean:  0.07389087535485556
separate: eco max 1.915034710474636
shared: eco max:  0.9348786384353573
shared: all mean:  0.07814717814292484
shared: all max:  1.6670768334516914

